# Libraries

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from typing import List, Optional
from pydantic import BaseModel, Field
from tqdm import tqdm
from langchain_core.caches import InMemoryCache


# Load .env


In [2]:
load_dotenv()

True

# Setting Langsmith tracing

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "Rag_travel_planner_v1.0"

# Load data Dfs

In [ ]:
landmark_prices = pd.read_csv(os.path.abspath('../data/egypt_v0.1.csv'))
places_api_data = pd.read_csv(os.path.abspath('../data/places_details.csv'))


# Store Dfs in langchain Documents

In [5]:
documents = []

for _, row in landmark_prices.iterrows():
    text = f"""
    Governorate: {row['Governorate/City']}
    Site: {row['Place']}
    Egyptian Ticket: {row['Egyptian']} EGP
    Egyptian Student Ticket: {row['EgyptianStudent']} EGP
    Foreign Ticket: {row['Foreign']} EGP
    Foreign Student Ticket: {row['ForeignStudent']} EGP
    Visiting Times: {row['VisitingTimes']}
    """
    documents.append(Document(page_content=text, metadata={"source": 'landmark_prices'}))

In [6]:
for _, row in places_api_data.iterrows():
    text = f"""
    Place Name: {row['displayName.text']}
    Place Primary Type: {row['primaryTypeDisplayName.text']}
    Place Types: {row['types']}
    Place Price: {row['priceRange.endPrice.units']} EGP
    Place Price Level: {row['priceLevel']}
    Place Location: {row['formattedAddress']}
    Place Star Rating: {row['rating']}
    Place website: {row['websiteUri']}
    """
    documents.append(Document(page_content=text, metadata={"source": 'Places_api', 'Type': f"{row['primaryTypeDisplayName.text']}", 'city': f"{row['formattedAddress']}"}))
    

# Embedd Documents (text ---> vectors of numbers)

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


c:\Users\mosai\anaconda3\envs\run-W-GPU\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
path = Path('faiss_mpnetv2_v1.0')

if not path.exists():
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local('faiss_mpnetv2_v1.0')
    
else:
    vectorstore = FAISS.load_local('faiss_mpnetv2_v1.0', embeddings, allow_dangerous_deserialization=True)


retriever = vectorstore.as_retriever(search_kwargs={"k": 50})

# Instantiating the LLM model with Groq provider.

In [9]:
# ChatNVIDIA.BaseCache = InMemoryCache

# ChatNVIDIA.model_rebuild()

# llm_model = ChatNVIDIA(model="meta/llama-3.1-405b-instruct", temperature=0)
# llm_model = ChatNVIDIA(model="meta/llama-3.3-70b-instruct", temperature=0)
# llm_model = ChatNVIDIA(model="microsoft/phi-4-mini-instruct", temperature=0)

In [10]:
# llm_model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq", temperature=0)
# llm_model = init_chat_model('deepseek-r1-distill-llama-70b', model_provider='groq', temperature=0)
llm_model = init_chat_model("qwen-qwq-32b", model_provider='groq', temperature=0)
# llm_model = init_chat_model("llama-3.3-70b-specdec", model_provider="groq", temperature=0)


In [11]:
# llm_model = init_chat_model('gemini-2.0-flash', model_provider='google_genai', temperature=0)

# Prompt Template

In [12]:
prompt_template = PromptTemplate(
    input_variables=["context", "user_query", "favorite_places", "visitor_type", "num_days", "budget"],
    template="""You are a helpful travel planner AI.
Use the context below, which contains information about ticket prices, place descriptions, restaurant details, and art gallery information.

Context:
{context}

User Query:
{user_query}

Additional Preferences:
- Favorite types of places: {favorite_places}
- Visitor type: {visitor_type} (e.g., Egyptian, Egyptian student, Foreign, or foreign student)
- Number of travel days: {num_days}
- Overall budget for all days: {budget} EGP
- Exclude hotels from the plan.
- Ensure that the itinerary includes at least 3 meals per day.

Based on the above, return a detailed {num_days}-day travel itinerary with approximate costs and suggestions. If some details are missing, make reasonable assumptions and indicate them.
"""
)


# Structured Output

In [13]:
json_schema = {
    "title": "TravelItinerary",
    "description": "A structured travel itinerary for the user.",
    "type": "object",
    "properties": {
        "days": {
            "type": "array",
            "description": "List of days with planned activities.",
            "items": {
                "type": "object",
                "properties": {
                    "day": {"type": "string", "description": "Theme of the Day or Day label, e.g., 'Day 1'"},
                    "activities": {
                        "type": "array",
                        "des3                                           cription": "Activities planned for the day.",
                        "items": {
                            "type": "object",
                            "properties": {
                                "time": {"type": "string", "description": "Time of the activity"},
                                "activity": {"type": "string", "description": "Name of the activity"},
                                "location": {"type": "string", "description": "Location name"},
                                "price_range": {"type": "string", "description": "Price range or cost"},
                            },
                            "required": ["time", "activity", "location"]
                        }
                    },
                    "approximate_cost": {"type": "string", "description": "Total cost for the day"}
                },
                "required": ["day", "activities", "approximate_cost"]
            }
        },
        "total_approximate_cost": {
            "type": "string",
            "description": "Total cost for the trip"
        },
        "notes": {
            "type": "string",
            "description": "Any additional notes or assumptions"
        }
    },
    "required": ["days", "total_approximate_cost"]
}

In [14]:
class Activity(BaseModel):
    time: str = Field(..., description="Time of the activity")
    activity: str = Field(..., description="Name of the activity")
    location: str = Field(..., description="Location name")
    price_range: Optional[str] = Field(None, description="Price range or cost")

class DayPlan(BaseModel):
    day: str = Field(..., description="Theme of the Day or Day label, e.g., 'Day 1'")
    activities: List[Activity] = Field(..., description="Activities planned for the day")
    approximate_cost: str = Field(..., description="Total cost for the day")

class TravelItinerary(BaseModel):
    days: List[DayPlan] = Field(..., description="List of days with planned activities")
    total_approximate_cost: str = Field(..., description="Total cost for the trip")
    notes: Optional[str] = Field(None, description="Any additional notes or assumptions")

In [15]:
structured_llm = llm_model.with_structured_output(json_schema)


# Generate a structured travel plan 

In [16]:

def generate_travel_plan(user_query, favorite_places, visitor_type, num_days, budget):
    docs = retriever.invoke(user_query)
    context_text = "\n".join([doc.page_content for doc in docs])
    prompt = prompt_template.format(
        context=context_text,
        user_query=user_query,
        favorite_places=favorite_places,
        visitor_type=visitor_type,
        num_days=num_days,
        budget=budget
    )
    response = llm_model.invoke(prompt)
    return response


In [17]:
user_query = "Plan a 3-day trip in Luxor with visits to cultural sites, art galleries, and dining(restaurents) options."
favorite_places = "Cultural sites, historical landmarks, art galleries"
visitor_type = "Foreign"  # or "Egyptian", "Egyptian student", "foreign student"
num_days = "3"
budget = "5000"  # Overall budget in EGP


In [18]:
travel_plan = generate_travel_plan(user_query, favorite_places, visitor_type, num_days, budget)
print(travel_plan)


content='\n<think>\nOkay, let\'s tackle this user\'s request. They want a 3-day trip in Luxor focusing on cultural sites, art galleries, and dining. They\'re a foreign visitor with a budget of 5000 EGP, excluding hotels. Need to include at least 3 meals a day.\n\nFirst, I need to go through the provided context to find the relevant places. The user\'s favorite types are cultural sites, historical landmarks, and art galleries. So I should prioritize those. Let\'s see the sites listed under Governorate: Luxor. There\'s Luxor Museum, Karnak Temples, Luxor Temple, Valley of the Kings, Temple of Hatshepsut, etc. Also, the art galleries like Luxor Art Gallery and Nobles Art Gallery.\n\nStarting with Day 1. Maybe start with the big attractions. Luxor Temple is a must. The ticket for Luxor Temple for foreigners is 500 EGP. Then Karnak Temple, but wait, the user mentioned Sphinx Avenue which requires tickets for both Karnak and Luxor Temples. Wait, the Sphinx Avenue entry requires purchasing th